In [71]:
import ipyleaflet
import json
import pandas as pd
import os
import requests
import geopandas as gpd
from ipywidgets import link, FloatSlider
from branca.colormap import linear

def load_data(url, filename, file_type):
    r = requests.get(url)
    with open(filename, 'w') as f:
        f.write(r.content.decode("utf-8"))
    with open(filename, 'r') as f:
        return file_type(f)

geo_json_data = load_data(
    'https://raw.githubusercontent.com/jupyter-widgets/ipyleaflet/master/examples/us-states.json',
    'us-states.json',
     json.load)

unemployment = load_data(
    'https://raw.githubusercontent.com/jupyter-widgets/ipyleaflet/master/examples/US_Unemployment_Oct2012.csv',
    'US_Unemployment_Oct2012.csv',
     pd.read_csv)

unemployment =  dict(zip(unemployment['State'].tolist(), unemployment['Unemployment'].tolist()))

covid = gpd.read_file('COVID19_Cases_US/COVID19_Cases_US.shp')

#Step 1
covid = covid[covid.Province_S != "District of Columbia"]
covid = covid[covid.Province_S != "Northern Mariana Islands"]

#Step 2
#I'm also including Province_S because we are grouping by state as the name of the variable, even though it doesn't
#say to do that in the instructions.
group_by_state = covid.loc[:, ['Confirmed', 'Province_S']].groupby('Province_S')
group_by_state_sum = group_by_state.sum()
print(group_by_state_sum)

#Step 3
state_id = []
for i in range(len(geo_json_data['features'])):
    state_id.append(geo_json_data['features'][i]['id'])

covid =  dict(zip(state_id, group_by_state_sum.Confirmed.values.tolist()))

layer = ipyleaflet.Choropleth(
    geo_data=geo_json_data,
    choro_data=covid,
    colormap=linear.YlOrRd_04,
    border_color='black',
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'})

m = ipyleaflet.Map(center = (43,-100), zoom = 4)
m.add_layer(layer)
m

                Confirmed
Province_S               
Alabama              2499
Alaska                226
Arizona              3036
Arkansas             1077
California          19031
Colorado             5655
Connecticut          8781
Delaware             1116
Florida             15698
Georgia             10206
Hawaii                435
Idaho                1232
Illinois            15078
Indiana              5943
Iowa                 1145
Kansas               1051
Kentucky             1341
Louisiana           17030
Maine                 537
Maryland             5529
Massachusetts       16790
Michigan            20346
Minnesota            1154
Mississippi          2003
Missouri             3432
Montana               332
Nebraska              519
Nevada               2318
New Hampshire         788
New Jersey          47437
New Mexico            973
New York           151079
North Carolina       3576
North Dakota          251
Ohio                 5148
Oklahoma             1526
Oregon      

Map(center=[43, -100], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…